In [1]:
path = "C:\\Users\\rocha\\Downloads\\e3c\\"
import xml.etree.ElementTree as ET
import os, pandas as pd
import json
import re
from bs4 import BeautifulSoup
import random
random.seed(17)

In [24]:
def file_name(file_dir):
    Lann=[]
    for root, dirs, files in os.walk(file_dir):
        for file in files:
            if file.endswith(".xml"):
                Lann.append(file)            
    return Lann

files = file_name(path+"layer1\\")
train= ['EN100017.xml', 'EN100022.xml', 'EN100024.xml', 'EN100029.xml', 'EN100090.xml', 'EN100156.xml', 'EN100184.xml', 'EN100265.xml', 'EN100310.xml', 'EN100399.xml', 'EN100415.xml', 'EN100427.xml', 'EN100432.xml', 'EN100466.xml', 'EN100497.xml', 'EN100593.xml', 'EN100605.xml', 'EN100645.xml', 'EN100655.xml', 'EN100700.xml', 'EN100705.xml', 'EN103007.xml', 'EN103220.xml', 'EN103266.xml', 'EN103442.xml', 'EN104184.xml', 'EN104655.xml', 'EN104891.xml', 'EN105223.xml', 'EN105832.xml', 'EN106156.xml', 'EN106489.xml', 'EN107405.xml', 'EN107424.xml', 'EN108254.xml', 'EN108798.xml']
dev = random.sample(train, round(len(train) * 0.2))
test = [item for item in files if item not in train]
train = [item for item in train if item not in dev]
print(len(files))
print(len(train))
print(len(dev))
print(len(test))

84
29
7
48


In [29]:
def xml2json(split):
    global doctimerels
    global timex3Classes
    global reltypes
    global treltypes
    nevents=0
    jsons=[]
    for i, fname in enumerate(files):
        if fname not in split: continue
#         print(fname)
        events = {}
        timexes={}
        rels = {}
        tlinkid2sourceid={}
        try:
            xml_dom = BeautifulSoup(open(path + "layer1\\"+fname, 'r', encoding='UTF-8'), 'xml')
        except:
            print(f"Can't load this file: {fname}")
        text = str(xml_dom.find("cas:Sofa")['sofaString'])
        eventxmls = xml_dom.find_all("custom:EVENT")
        timexmls = xml_dom.find_all("custom:TIMEX3")
        etlinks = xml_dom.find_all("custom:EVENTTLINKLink")
        ttlinks = xml_dom.find_all("custom:TIMEX3TimexLinkLink")
        for event in eventxmls:
            # <custom:EVENT xmi:id="1505" sofa="12" begin="25" end="34" TLINK="1839" contextualAspect="N/A" contextualModality="ACTUAL" degree="N/A" docTimeRel="CONTAINS" eventType="N/A" permanence="FINITE" polarity="POS"/>
            eid = event["xmi:id"]
            permanence = ""
            TLINKs= []
            try:
                permanence=event['permanence'] # permanent/finite
            except:pass
            try:
                TLINKs= event['TLINK'].split(" ")
                for tlink in TLINKs:
                    if tlink in tlinkid2sourceid:
                        tlinkid2sourceid[tlink].append(eid)
                    else:
                        tlinkid2sourceid[tlink]=[eid]
            except: pass
            start =event['begin']
            contextualAspect=event['contextualAspect'] # intermittent, novel, ma
            contextualModality=event['contextualModality'] # actual, hedged, hypothetical, generic,
            degree=event["degree"] # most/little/na
            docTimeRel=event['docTimeRel'] # (before, after, contains, is-contained, overlap)
            end=event['end']
            eventType=event['eventType'] # aspectual/evidential/n/a,
            polarity=event["polarity"] # pos, neg
            if docTimeRel in doctimerels:
                doctimerels[docTimeRel]+=1
            else:
                doctimerels[docTimeRel]=1
            events[eid] = {'start':start, 'end':end, 'text': text[int(start):int(end)], 'type': eventType, 'contextualAspect':contextualAspect, 
                          'contextualModality':contextualModality, 'TLINKs':TLINKs, 'degree':degree, 'docTimeRel':docTimeRel,
                          'end':end, 'eventType': eventType, 'permanence':permanence, 'polarity':polarity}

        for timex in timexmls:
    #       <custom:TIMEX3 xmi:id="9442" sofa="12" begin="130" end="135" functionInDocument="NONE" timex3Class="DURATION" timexLink="10171 10174 10177 10180 10183" value="P4D"/>
            eid = timex["xmi:id"]
            value = ""
            functionInDocument = ""
            TLINKs = []
            try:
                functionInDocument=timex['functionInDocument'] # intermittent, novel, ma
            except: pass
            try:
                TLINKs= timex['timexLink'].split(" ")
                for tlink in TLINKs:
                    if tlink in tlinkid2sourceid:
                        tlinkid2sourceid[tlink].append(eid)
                    else:
                        tlinkid2sourceid[tlink]=[eid]
            except: pass
            try:
                value=timex["value"] # most/little/na   
            except: pass
            start= timex['begin']
            end =timex['end']

            timex3Class=timex['timex3Class'] # actual, hedged, hypothetical, generic,
            if timex3Class in timex3Classes:
                timex3Classes[timex3Class]+=1
            else:
                timex3Classes[timex3Class]=1
            timexes[eid] = {'start':start, 'end':end, 'text': text[int(start):int(end)], 'TLINKs':TLINKs, 'type': timex3Class, 
                            'functionInDocument':functionInDocument, 'value': value, }

        for etlink in etlinks:
            # <custom:EVENTTLINKLink xmi:id="1839" role="OVERLAP" target="1550"/>
            rid = etlink['xmi:id']
            reltype = etlink['role']
            target = etlink['target']
            source = tlinkid2sourceid[rid][0]
            if source ==target:
                try:
                    source = tlinkid2sourceid[rid][1]
                except:
                    print(f"source and target same in etlink:{rid}")
                    continue
            if reltype in reltypes:
                reltypes[reltype]+=1
            else:
                reltypes[reltype]=1
            rels[rid] = {'rid':rid, 'reltype':'elink', 'type':reltype, 'head':source, 'tail':target}

        for ttlink in ttlinks:
            # <custom:TIMEX3TimexLinkLink xmi:id="5089" role="CONTAINS" target="3920"/>
            rid = ttlink['xmi:id']
            reltype = ttlink['role']
            target = ttlink['target']
            source = tlinkid2sourceid[rid][0]
            if source==target:
                try:
                    source = tlinkid2sourceid[rid][1]
                except:
                    print(f"source and target same in ttlink:{rid}")
                    continue
            if reltype in treltypes:
                treltypes[reltype]+=1
            else:
                treltypes[reltype]=1
            rels[rid] = {'rid':rid, 'reltype':'tlink', 'type':reltype, 'head':source, 'tail':target}

        data = {"file_id": fname.split(".")[0], "text":text, "entities":events, 'timexes':timexes, "relations":rels}
        jsons.append(data)
        nevents+=len(events.keys())
    print(f"events:{nevents}")
    print(f"doctimerels:{doctimerels}")
    print(f"timex3Classes: {timex3Classes}")
    print(f"reltypes: {reltypes}")
    print(f"treltypes: {treltypes}")
    return jsons

In [30]:
for splitname, split in [("test", test), ("dev", dev), ("train", train)]: 
    print()
    print(splitname)
    doctimerels ={}
    timex3Classes={}
    reltypes = {}
    treltypes = {}
    data = xml2json(split)
    print(len(data))
    with open(f'{path}{splitname}.json', 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)


test
events:2843
doctimerels:{'IS-CONTAINED': 446, 'BEFORE': 2270, 'CONTAINS': 95, 'OVERLAP': 25, 'AFTER': 7}
timex3Classes: {'DATE': 1, 'PREPOSTEXP': 1, 'DURATION': 1, 'QUANTIFIER': 1, 'SET': 1}
reltypes: {'BEFORE': 707, 'CONTAINS': 1022, 'OVERLAP': 416, 'ENDS-ON': 43, 'BEGINS-ON': 83, 'SIMULTANEOUS': 204}
treltypes: {'BEGINS-ON': 36, 'ENDS-ON': 49, 'OVERLAP': 13, 'CONTAINS': 115, 'BEFORE': 29, 'SIMULTANEOUS': 31}
48

dev
events:337
doctimerels:{'IS-CONTAINED': 170, 'BEFORE': 167}
timex3Classes: {'DATE': 1, 'SET': 1, 'QUANTIFIER': 1, 'DURATION': 1}
reltypes: {'CONTAINS': 128, 'BEFORE': 64, 'ENDS-ON': 5, 'SIMULTANEOUS': 22, 'BEGINS-ON': 15, 'OVERLAP': 66}
treltypes: {'SIMULTANEOUS': 3, 'CONTAINS': 8, 'OVERLAP': 5, 'BEGINS-ON': 8, 'ENDS-ON': 7, 'BEFORE': 2}
7

train
source and target same in etlink:6100
events:1705
doctimerels:{'BEFORE': 1376, 'IS-CONTAINED': 292, 'OVERLAP': 7, 'CONTAINS': 29, 'AFTER': 1}
timex3Classes: {'DURATION': 1, 'DATE': 1, 'TIME': 1, 'SET': 1, 'QUANTIFIER': 1, '